In [1]:
import pandas as pd
import  numpy as np
import pymongo
from pymongo import MongoClient
from time import gmtime, strftime
from datetime import datetime



# Current Date

In [2]:
date = datetime.now().strftime('%Y-%m-%d')
date

'2020-06-12'

# https://www.coronatracker.com/country/india/

In [3]:
df = pd.read_json("https://api.coronatracker.com/v4/analytics/newcases/country?countryCode=IN&startDate=2020-04-10&endDate=2020-06-10")

In [4]:
df.tail()

,country,country_code,last_updated,new_deaths,new_infections,new_recovered
55,India,IN,2020-06-05T00:01:00.000Z,286,9471,4781
56,India,IN,2020-06-06T00:02:00.000Z,297,10438,5464
57,India,IN,2020-06-07T00:04:00.000Z,261,10884,5153
58,India,IN,2020-06-08T00:05:00.000Z,266,8422,5247
59,India,IN,2020-06-09T00:03:00.000Z,277,10218,5572


# Saving India data in csv file

In [5]:
India = df.tail(1)
India
India.to_csv(date + "_India.csv")

# https://www.covid19india.org/

In [6]:
df2 = pd.read_json("https://api.covid19india.org/v3/min/data.min.json")


In [7]:
Maharashtra = [s for s in df2["MH"] if "tested" in s]
Maharashtra

[{'last_updated': '2020-06-12T20:44:48+05:30',
  'tested': {'last_updated': '2020-06-12',
   'source': 'https://twitter.com/Maha_MEDD/status/1271316085543849986?s=19'}},
 {'confirmed': 101141, 'deceased': 3717, 'recovered': 47796, 'tested': 626521},
 {'confirmed': 3493, 'deceased': 127, 'recovered': 1718, 'tested': 15731}]

# Maharashtra New cases 

In [10]:


New_cases_in_Maharashtra = Maharashtra[2]["confirmed"]
Recoveries_in_Maharashtra =  Maharashtra[2]["recovered"]
Deaths_in_Maharashtra = Maharashtra[2]["deceased"]
Tested_in_Maharashtra = Maharashtra[2]["tested"]
Maharashtra_New = [New_cases_in_Maharashtra,Recoveries_in_Maharashtra,Deaths_in_Maharashtra,Tested_in_Maharashtra]
Maharashtra_New
Maharashtra_New = pd.DataFrame(Maharashtra_New)
Maharashtra_New  = Maharashtra_New.transpose()
Maharashtra_New.columns = ["New_cases_in_Maharashtra","Recoveries_in_Maharashtra","Deaths_in_Maharashtra","Tested_in_Maharashtra"]
Maharashtra_New.head()



,New_cases_in_Maharashtra,Recoveries_in_Maharashtra,Deaths_in_Maharashtra,Tested_in_Maharashtra
0,3493,1718,127,15731


# Maharashtra  cases 

In [11]:


Cases = Maharashtra[1]["confirmed"]
Recoveries =  Maharashtra[1]["recovered"]
Deaths = Maharashtra[1]["deceased"]
Tested = Maharashtra[1]["tested"]
Maharashtra = [date,Cases,Recoveries,Deaths,Tested]
Maharashtra
Maharashtra = pd.DataFrame(Maharashtra)
Maharashtra  = Maharashtra.transpose()
Maharashtra.columns = ["Date","Cases","Recoveries","Deaths","Tested"]
Maharashtra.head()



,Date,Cases,Recoveries,Deaths,Tested
0,2020-06-12,101141,47796,3717,626521


In [12]:
Karnataka = [s for s in df2["KA"] if "tested" in s]
Karnataka

[{'last_updated': '2020-06-12T19:14:49+05:30',
  'notes': 'Two active cases in KA reduced as these confirmed patients died of non-CoViD19 related reasons',
  'tested': {'last_updated': '2020-06-12',
   'source': 'https://covid19.karnataka.gov.in/storage/pdf-files/Media-Bulletin/12-06-2020%20HMB%20English.pdf'}},
 {'confirmed': 6516,
  'deceased': 79,
  'migrated': 2,
  'recovered': 3440,
  'tested': 426341},
 {'confirmed': 271, 'deceased': 7, 'recovered': 464, 'tested': 9835}]


# Karnataka New cases 

In [15]:


New_cases_in_Karnataka = Karnataka[2]["confirmed"]
Recoveries_in_Karnataka =  Karnataka[2]["recovered"]
Deaths_in_Karnataka = Karnataka[2]["deceased"]
Tested_in_Karnataka = Karnataka[2]["tested"]
Karnataka_New = [New_cases_in_Karnataka,Recoveries_in_Karnataka,Deaths_in_Karnataka,Tested_in_Karnataka]
Karnataka_New
Karnataka_New = pd.DataFrame(Karnataka_New)
Karnataka_New  = Karnataka_New.transpose()
Karnataka_New.columns = ["New_cases_in_Karnataka","Recoveries_in_Karnataka","Deaths_in_Karnataka","Tested_in_"]
Karnataka_New.head()



,New_cases_in_Karnataka,Recoveries_in_Karnataka,Deaths_in_Karnataka,Tested_in_
0,271,464,7,9835



# Karnataka  cases 

In [16]:


Cases = Karnataka[1]["confirmed"]
Recoveries =  Karnataka[1]["recovered"]
Deaths = Karnataka[1]["deceased"]
Tested = Karnataka[1]["tested"]
Karnataka = [Cases,Recoveries,Deaths,Tested]
Karnataka
Karnataka = pd.DataFrame(Karnataka )
Karnataka  = Karnataka .transpose()
Karnataka.columns = ["cases","recoveries","deaths","tested"]
Karnataka.head()



,cases,recoveries,deaths,tested
0,6516,3440,79,426341


In [17]:
df = pd.concat([Maharashtra,Maharashtra_New,Karnataka,Karnataka_New],axis =1)

In [18]:
df

,Date,Cases,Recoveries,Deaths,Tested,New_cases_in_Maharashtra,Recoveries_in_Maharashtra,Deaths_in_Maharashtra,Tested_in_Maharashtra,cases,recoveries,deaths,tested,New_cases_in_Karnataka,Recoveries_in_Karnataka,Deaths_in_Karnataka,Tested_in_
0,2020-06-12,101141,47796,3717,626521,3493,1718,127,15731,6516,3440,79,426341,271,464,7,9835


# Saving Maharashtra & Karnataka data in csv file

In [19]:
df = pd.concat([Maharashtra,Maharashtra_New,Karnataka,Karnataka_New],axis =1)
df
df.to_csv(date + "_State.csv",index = False)

# uploading state data in Mongodb

In [20]:
client = MongoClient('localhost', 27017)
db = client.covid
state = db.state
state = pd.read_csv(date + "_State.csv") #csv file which you want to import
records_ = state.to_dict(orient = 'records')
result = db.state.insert_many(records_ )

# uploading India data in Mongodb

In [21]:
client = MongoClient('localhost', 27017)
db = client.covid
India = db.India
India = pd.read_csv(date + "_India.csv") #csv file which you want to import
records_ = India.to_dict(orient = 'records')
result = db.India.insert_many(records_ )